In [1]:
import pandas as pd
from ast import literal_eval

In [2]:
artists = ['Frank Sinatra', 'The Beatles', 'Michael Jackson', 'Eminem', 'Rihanna']

In [3]:
import re
import unidecode

def remove_superflous(text):
    text = re.sub(r'\([^)]*\)', '', text).split('-')[0]
    text = unidecode.unidecode(text)
    text = text.lower().replace("'", "").replace("’", "")\
                        .replace("‘","").replace('/', '')\
                        .replace("\"", "").replace(".","")\
                        .replace("?", "").replace("!", "")\
                        .replace("&", "and").replace(':','')\
                        .replace(",", "").replace('  ', ' ')
    while text[0] == ' ':
        text = text[1:]
    while text[-1] == ' ':
        text = text[:-1]

    text = re.sub(r'\d+$', '', text)
    
    return text.replace(' ', '_')
                             
#Load all lyrics
lyrics = pd.read_csv('../data/full_lyrics_df.csv').drop('Unnamed: 0', axis=1)

#Load all relevant musics
music = pd.read_csv('../data/full_spotify_df.csv').drop('Unnamed: 0', axis=1)
music.artists_album = music.artists_album.map(lambda artists: literal_eval(artists)[0]['name'])
music = music[music.artists_album.isin(artists)]

music['clean_title'] = music.name_track.map(lambda track: remove_superflous(track))
lyrics['clean_title'] = lyrics.title.map(lambda title: remove_superflous(title) if title != '313' and title != '1999' else title)
#MJ track Shoo-Be-Doo-Be-Doo-Da-Day gets changed for shoo in the clean title category; not a problem as there are no other Shoo songs

music = music[~(music.clean_title.map(lambda title: title.lower()).str.contains('mix'))]
music = music[~(music.name_album == 'XSCAPE - Track by Track Commentary')]
music = music[~(music.name_track.map(lambda track: track.lower()).str.contains('mix'))]
music = music[~(music.name_track.map(lambda track: track.lower()).str.contains('monologue'))]
music = music[~(music.name_track.map(lambda track: track.lower()).str.contains('segment'))]
music = music[~(music.name_track.map(lambda track: track.lower()).str.contains('announcement'))]
music = music[~(music.name_track.map(lambda track: track.lower()).str.contains('presentation'))]
music = music[~(music.name_track.map(lambda track: track.lower()).str.contains('medley'))]
music = music[~(music.name_track.map(lambda track: track.lower()).str.contains('remarks'))]
music = music[~(music.name_track.map(lambda track: track.lower()).str.contains('introduction'))]
music = music[~(music.name_track.map(lambda track: track.lower()).str.contains('speak'))]
music = music[~(music.name_track.map(lambda track: track.lower()).str.contains('welcome'))]
music = music[~(music.name_track.map(lambda track: track.lower()).str.contains('skit'))]
music = music[~(music.name_track.map(lambda track: track.lower()).str.contains('interview'))]
music = music[~(music.name_track.map(lambda track: track.lower()).str.contains('excerpt'))]
music = music[~(music.name_album == "Selections From A Voice On Air (1935-1955)")]

music.loc[(music.clean_title.map(lambda title: title) == 'i_get_along_with_you_very_well'), 'clean_title'] = 'i_get_along_without_you_very_well'
music.loc[(music.clean_title.map(lambda title: title) == 'lady_is_a_tramp'), 'clean_title'] = 'the_lady_is_a_tramp'
music.loc[(music.clean_title.map(lambda title: title) == 'play_on_clarke_street'), 'clean_title'] = 'clarke_street'
music.loc[(music.clean_title.map(lambda title: title) == 'play_off_the_tender_trap'), 'clean_title'] = 'the_tender_trap'
music.loc[(music.clean_title.map(lambda title: title) == 'bows_the_tender_trap'), 'clean_title'] = 'the_tender_trap'
music.loc[(music.clean_title.map(lambda title: title) == 'how_do_you_keep_the_music_playing_'), 'clean_title'] = 'how_do_you_keep_the_music_playing'

music.loc[(music.name_track.map(lambda track: track.lower()).str.contains('\*')), 'clean_title'] = 'asshole'


lyrics = lyrics[~(lyrics.clean_title.map(lambda title: title.lower()).str.contains('mix'))]
lyrics = lyrics.append(lyrics[(lyrics.clean_title == 'it_came_upon_the_midnight_clear')])
lyrics.loc[lyrics.clean_title == 'it_came_upon_the_midnight_clear', 'clean_title'] = ['it_came_upon_the_midnight_clear', 'it_came_upon_a_midnight_clear']
lyrics = lyrics.append(lyrics[(lyrics.clean_title == 'day_in')])
lyrics.loc[lyrics.clean_title == 'day_in', 'clean_title'] = ['day_in', 'day_in_day_out']
lyrics = lyrics.append(lyrics[(lyrics.clean_title == 'you_brought_a_new_kind_of_love_to_me')])
lyrics.loc[lyrics.clean_title == 'you_brought_a_new_kind_of_love_to_me', 'clean_title'] = ['you_brought_a_new_kind_of_love_to_me', 'you_brought_a_new_kind_of_love']
lyrics = lyrics.append(lyrics[(lyrics.clean_title == 'hey_look_no_crying')])
lyrics.loc[lyrics.clean_title == 'hey_look_no_crying', 'clean_title'] = ['hey_look_no_crying', 'hey_look_no_cryin']
lyrics.loc[lyrics.clean_title == 'one_hundred_years_from_today', 'clean_title'] = ['a_hundred_years_from_today']
lyrics.loc[lyrics.clean_title == 'song_is_you', 'clean_title'] = ['the_song_is_you']
lyrics.loc[lyrics.clean_title == 'when_im_sixty', 'clean_title'] = ['when_im_sixty_four', 'when_im_sixty', 'when_im_sixty']

In [4]:
lyrics.columns

Index(['artist_id', 'artist_name', 'featured_artists', 'featured_video',
       'full_title', 'header_image_thumbnail_url', 'header_image_url',
       'lyrics', 'lyrics_state', 'release_date', 'release_date_for_display',
       'song_art_image_thumbnail_url', 'song_art_image_url', 'title',
       'title_with_featured', 'clean_title'],
      dtype='object')

In [5]:
missing_titles = set(music.clean_title.unique()) - (set(music.clean_title.unique()) & (set(lyrics.clean_title.unique())))
len(missing_titles)

40

### Missing lyrics:

- Leave it all to me - Sinatra
- Hush-A-Bye Island - Sinatra
- This Can't Be Love - Sinatra
- Watching Me - MJ
- Torture - MJ
- Nice Guy - Eminem
- When Joanna Loved Me - Sinatra
### Others
- Yellow Submarine: instrumentals
- Live At The BBC (1 and 2): interviews
- Trilogy: Past, Present...; live at meadowlands; Concierto;   : instrumental

There are 110 songs in the music DF that find no match in the lyrics df.

### Merge

In [6]:
lyrics = lyrics.drop_duplicates(subset='clean_title')[['clean_title', 'lyrics', 'lyrics_state']]

In [7]:
full_df = music.merge(lyrics, how='left', on='clean_title')
full_df.head()

,Unnamed: 0.1,name_track,duration_ms,explicit,artists_track,artists_track_uri,featuring,track_number,disc_number,name_album,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,clean_title,lyrics,lyrics_state
0,0,Nice 'N' Easy - Remastered,165786,False,['Frank Sinatra'],['spotify:artist:1Mxqyy3pSjf8kZZL4QVxS0'],False,1,1,Sinatra Sings Alan & Marilyn Bergman,...,0.0327,0.876,0.000000,0.111,0.471,109.937,4,nice_n_easy,Let's take it nice and easy\nIt's gonna be so ...,complete
1,1,Ol' Macdonald,160306,False,['Frank Sinatra'],['spotify:artist:1Mxqyy3pSjf8kZZL4QVxS0'],False,2,1,Sinatra Sings Alan & Marilyn Bergman,...,0.0375,0.738,0.000000,0.181,0.730,84.750,4,ol_macdonald,"Ol' MacDonald had a farm, E-I-E-I-O\nAnd on th...",complete
2,2,Sentimental Baby,156280,False,['Frank Sinatra'],['spotify:artist:1Mxqyy3pSjf8kZZL4QVxS0'],False,3,1,Sinatra Sings Alan & Marilyn Bergman,...,0.0294,0.899,0.000501,0.169,0.285,82.298,3,sentimental_baby,"Sentimental baby, sweet and gentle baby\nJust ...",complete
3,3,Love Looks So Well On You,158893,False,['Frank Sinatra'],['spotify:artist:1Mxqyy3pSjf8kZZL4QVxS0'],False,4,1,Sinatra Sings Alan & Marilyn Bergman,...,0.0317,0.901,0.000457,0.107,0.201,85.471,4,love_looks_so_well_on_you,"Like moonlight looks on a country side, like a...",complete
4,4,Sleep Warm,166066,False,['Frank Sinatra'],['spotify:artist:1Mxqyy3pSjf8kZZL4QVxS0'],False,5,1,Sinatra Sings Alan & Marilyn Bergman,...,0.0320,0.929,0.014200,0.110,0.139,88.004,5,sleep_warm,"Sleep warm, sleep tight, when you turn off the...",complete


In [8]:
#is_it_scary_lyrics = open('../processbook/missing_lyrics/is_it_scary.txt', 'r').read()
watching_me_lyrics = open('missing_lyrics/watching_me.txt', 'r').read()
torture_lyrics = open('missing_lyrics/torture.txt', 'r').read()
leave_it_all_lyrics = open('missing_lyrics/leave_it_all.txt', 'r').read()
hush_a_bye_lyrics = open('missing_lyrics/hush_a_bye.txt', 'r').read()
this_cant_be_love_lyrics = open('missing_lyrics/this_cant_be_love.txt', 'r').read()
nice_guy_lyrics = open('missing_lyrics/nice_guy.txt', 'r').read()
joanna_love_lyrics = open('missing_lyrics/joanna_love.txt', 'r').read()
you_da_one_lyrics = open('missing_lyrics/you_da_one.txt', 'r').read()

In [9]:
full_df.loc[full_df.clean_title.str.contains('watching_me'), 'lyrics'] = watching_me_lyrics
full_df.loc[full_df.clean_title.str.contains('torture'), 'lyrics'] = torture_lyrics
full_df.loc[full_df.clean_title.str.contains('leave_it_all'), 'lyrics'] = leave_it_all_lyrics
full_df.loc[full_df.clean_title == 'hush', 'lyrics'] = hush_a_bye_lyrics
full_df.loc[full_df.clean_title == 'nice_guy', 'lyrics'] = nice_guy_lyrics
full_df.loc[full_df.clean_title.str.contains('joanna'), 'lyrics'] = joanna_love_lyrics
full_df.loc[full_df.clean_title.str.contains('this_cant'), 'lyrics'] = this_cant_be_love_lyrics
full_df.loc[full_df.clean_title.str.contains('you_da_one'), 'lyrics'] = you_da_one_lyrics

In [49]:
missing_lyrics = full_df[(full_df.drop('lyrics_state', axis=1).isnull().any(axis=1))][['name_track', 'name_album', 'artists_album', 'clean_title', 'artists_track']]


In [50]:
full_df.to_csv('../data/spot_lyr_df.csv')

In [51]:
full_df.columns

Index(['Unnamed: 0.1', 'name_track', 'duration_ms', 'explicit',
       'artists_track', 'artists_track_uri', 'featuring', 'track_number',
       'disc_number', 'name_album', 'artists_album', 'total_tracks',
       'uri_track', 'uri_album', 'danceability', 'energy', 'key', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'time_signature', 'clean_title', 'lyrics',
       'lyrics_state'],
      dtype='object')

In [52]:
missing_lyrics.to_csv('../data/missing_lyrics.csv')

In [53]:
missing_lyrics.count()

name_track       96
name_album       96
artists_album    96
clean_title      96
artists_track    96
dtype: int64

# Find missing lyrics

In [81]:
missing_lyrics_df = missing_lyrics.copy()

missing_lyrics_df.reset_index(drop=True, inplace=True)

missing_lyrics_df['clean_title'] = missing_lyrics_df['clean_title'].str.replace("_", " ")

# for some reasons, the 'artists_track' column is now a "string":: "['Frank Sinatra']". Convert it back to list.

import ast
missing_lyrics_df['artists_track'] = missing_lyrics_df['artists_track'].apply(ast.literal_eval)

missing_lyrics_df.head(2)

import lyricsgenius

genius = lyricsgenius.Genius(
    "5y-Ynf87UKhSvEAB90B3eAT3P6Y-aiVft2qPYMVtyRVi85u4JhgOQQrXaJiTyJN0"
)

def get_lyrics(title, artist, genius=genius):
    song = genius.search_song(title, artist)
    
    if song:
        return song.lyrics
    else:
        return ""

# Get missing lyrics

def get_missing_lyrics(missing_lyrics_df):
    missing_lyrics_df['lyrics'] = ""

    for i, row in list(missing_lyrics_df.iterrows()):
        artist = row['artists_track'][0]
        title = row['clean_title']

        lyircs = get_lyrics(title, artist)

        if lyircs.strip() == "":
            print("Cannot find lyrics for ", artist, title)

        missing_lyrics_df.loc[i, 'lyrics'] = lyircs

    return missing_lyrics_df

Searching for "taking a chance on love" by Frank Sinatra...
Done.
Searching for "something" by Frank Sinatra...
Done.
Searching for "concierto de aranjuez" by Frank Sinatra...
No results found for: 'concierto de aranjuez Frank Sinatra'
Cannot find lyrics for  Frank Sinatra concierto de aranjuez
Searching for "bows you are there" by Frank Sinatra...
Done.
Searching for "the shadow of your smile" by Frank Sinatra...
Done.
Searching for "it was a very good year" by Frank Sinatra...
Done.
Searching for "bows my way" by Frank Sinatra...
Done.
Searching for "clarke street" by Frank Sinatra...
Done.
Searching for "april in paris" by Frank Sinatra...
Done.
Searching for "ninna nanna" by Domenico Modugno...
Done.
Searching for "i concentrate on you" by Frank Sinatra...
Done.
Searching for "april in paris" by Frank Sinatra...
Done.
Searching for "it was a very good year" by Frank Sinatra...
Done.
Searching for "taking a chance on love" by Frank Sinatra...
Done.
Searching for "it was a very good 

,name_track,name_album,artists_album,clean_title,artists_track,lyrics
0,Taking a Chance On Love - 2007 - Remaster,I've Got You Under My Skin,Frank Sinatra,taking a chance on love,[Frank Sinatra],Writer(s): Latouche/Fetter/Duke\n\nHere I go a...
1,"Something - Live At The Budokan Hall, Tokyo / ...",Sinatra In Japan (Live At The Budokan/1985),Frank Sinatra,something,[Frank Sinatra],"Something in the way she moves, attracts me li..."
2,Concierto de Aranjuez - Live At The Budokan Ha...,Sinatra In Japan (Live At The Budokan/1985),Frank Sinatra,concierto de aranjuez,[Frank Sinatra],
3,Bows: You Are There - Live At The Budokan Hall...,Sinatra In Japan (Live At The Budokan/1985),Frank Sinatra,bows you are there,[Frank Sinatra],[Instrumental]
4,The Shadow Of Your Smile - Live At The Sands H...,Standing Room Only,Frank Sinatra,the shadow of your smile,[Frank Sinatra],The shadow of your smile\nWhen you have gone\n...


In [85]:
missing_lyrics_df.to_csv("../data/additional_lyrics.csv", index=False)

### Import and insert additional lyrics

In [86]:
additional_lyrics_df = pd.read_csv("../data/additional_lyrics.csv")

full_df_additional = full_df.merge(additional_lyrics_df[['name_track', 'name_album', 'lyrics']], on=['name_track', 'name_album'])
full_df_additional.fillna("", inplace=True)

full_df_additional['lyrics'] = full_df_additional['lyrics_x'] + full_df_additional['lyrics_y']
full_df_additional.drop(columns=['lyrics_x', 'lyrics_y'], inplace=True)


full_df_additional

full_df.to_csv('../data/spot_lyr_df.csv')